Files have been disorganized and placed into wrong subfolders. This notebook is used to identify correct folders and fix wrong placements.

In [1]:
# import packages
import numpy as np
import cadquery as cq
from scipy.stats import qmc
import math
import itertools
import matplotlib.pyplot as plt
import os, re, shutil
import json
import copy
import random
import sys
import miniball
import pandas as pd
import glob
import contextlib

In [2]:
# load the param file that was used to create rosettes
params_path = '/glade/u/home/joko/ice3d/output/params_200_50.json'
# load saved json
with open(params_path, 'rb') as file:
    params_list = json.load(file)

In [4]:
'''
Creating a dictionary that holds the lists of ros ids 
in each stl sub-directory
'''
# Directory path
basepath = '/glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/'

# Get a list of all folders (directories) in the specified directory
folders = [f for f in os.listdir(basepath) if os.path.isdir(os.path.join(basepath, f))]
id_dict = {}
for f in folders:
    directory_path = os.path.join(basepath, f)
    print(f'directory: {directory_path}')
    # Get all .stl files, including those in subdirectories
    file_paths = glob.glob(directory_path + '**/*.stl', recursive=True)
    
    # Initialize an empty list to store the gen_ids
    ids = []
    
    # Regular expression to extract the number
    pattern = r'ros-test-(\d+)\.stl'
    
    # Loop through the files and extract the number
    for file in file_paths:
        match = re.search(pattern, file)
        if match:
            # Extract the number and append it to the list
            number = match.group(1)
            ids.append(int(number))  # Convert to integer if needed
    # add to dictionary
    id_dict[f] = ids
    
    # Print the list of gen_ids
    print(len(ids))

directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9
10010
directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/8
10010
directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/10
10006
directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/7
10009
directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/4
9999
directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/6
10013
directory: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/5
10014


In [23]:
# identify files in wrong folder
misplaced = {}
for i in folders:
    misplaced_ids = []
    ids = id_dict[i]
    print(f'folder {i}')
    for id in ids:
        params = params_list[id]
        n_arms = params[0][-1]
        if int(i) != n_arms:
            misplaced_ids.append(id)
    misplaced[i] = misplaced_ids

folder 9
folder 8
folder 10
folder 7
folder 4
folder 6
folder 5


In [27]:
for key in misplaced:
    print(f'folder {key}')
    print(len(misplaced[key]))

folder 9
10
folder 8
10
folder 10
6
folder 7
9
folder 4
0
folder 6
14
folder 5
14


In [33]:
# move to correct folder
for key in misplaced:
    print(f'folder {key}')
    src_dir = os.path.join(basepath, key)
    ids = misplaced[key]
    for i in ids:
        filename = f'ros-test-{i:06d}.stl'
        src_item = os.path.join(src_dir, filename)
        print(f'transferring {src_item}')
        params = params_list[i]
        correct_f = params[0][-1]
        dest_dir = os.path.join(basepath, f'{correct_f}')
        print(f'--> to correct destination: {dest_dir}')
        destination_file = os.path.join(dest_dir, os.path.basename(filename))
        # Move the file, overwrite if it already exists
        shutil.move(src_item, destination_file)

folder 9
transferring /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-000004.stl
--> to correct destination: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/4
transferring /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-000070.stl
--> to correct destination: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/4
transferring /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-000062.stl
--> to correct destination: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/4
transferring /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-000001.stl
--> to correct destination: /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/4
transferring /glade/derecho/scratch/joko/synth-ros/params_200_50-debug-20250316/stl/9/ros-test-000051.stl
--> to correct destination: /glade/derecho/scratch/joko/s